<a href="https://colab.research.google.com/github/Varun9213/SIIM-FISABIO-RSNA-COVID-19-Detection/blob/main/mmdetection_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle
!pip install albumentations==0.5.2 --no-cache-dir -q

     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 952kB 8.5MB/s 
     |████████████████████████████████| 37.2MB 1.4MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
!mkdir ~/.kaggle
!cp /content/gdrive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d sohomdey/siim-covid19-detection-512

 98% 802M/817M [00:04<00:00, 189MB/s]
100% 817M/817M [00:04<00:00, 191MB/s]


In [5]:
!mkdir train
!unzip -q /content/siim-covid19-detection-512.zip -d train
!kaggle datasets download -d kozodoi/timm-pytorch-image-models
!mkdir timm
!unzip -q /content/timm-pytorch-image-models.zip -d timm

 35% 5.00M/14.2M [00:00<00:00, 45.9MB/s]
100% 14.2M/14.2M [00:00<00:00, 69.2MB/s]


In [6]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [7]:
!pip install mmcv-full

!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

     |████████████████████████████████| 317kB 5.2MB/s 
     |████████████████████████████████| 194kB 8.6MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.9-cp37-cp37m-linux_x86_64.whl size=29395926 sha256=8a949ae0261628be08e578e6adea6a30a93404a21c10b5eaf8be151742a0b790
  Stored in directory: /root/.cache/pip/wheels/e0/2b/d5/410850bd0f29e241507daf354083142a0c25d66667f22e707a
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 19329, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 19329 (delta 114), reused 185 (delta 76), pack-reused 19062
Receiving objects: 100% (19329/19329), 23.61 MiB | 31.65 MiB/s, done.
Resolving deltas: 100% (13502/13502), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=6bd7176350a0b8d12cc4803110d2f875d136a9e7cb7888254f8f7337c7cfb426

In [1]:
# Check Pytorch installation
import torch, torchvision, os
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu102 True
2.14.0
11.0
GCC 7.5


**CONFIG**

In [2]:
class CFG:
  cls=['opacity']

  dim=512
  size=640
  fold=0
  debug=False
  seed=42
  generate_ann=False
  prefix_img = '/content/train/images/'

  if debug:
    prefix_ann = '/content/annotaions/'
    #!mkdir /content/annotaions
  else:
    prefix_ann = '/content/gdrive/MyDrive/kaggle/COVID_annotaions/'

  if generate_ann and not debug:
    !mkdir /content/gdrive/MyDrive/kaggle/COVID_annotaions/

  n_classes=len(cls)
  batch_size=4
  num_workers=4
  epochs=15
  if debug:
    epochs=1
  lr=3e-4
  min_lr=3e-5
  _base_='/content/mmdetection/configs/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py'
  checkpoint='https://openmmlab.oss-cn-hangzhou.aliyuncs.com/mmdetection/v2.0/vfnet/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth'


  if debug:
    outdir='/content/outdir/'
    !mkdir /content/outdir
  else:
    outdir='/content/gdrive/MyDrive/kaggle/'  

**Data Frames**

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold

if CFG.generate_ann:

  raw = pd.read_csv('/content/train/train.csv')
  raw['n_boxes'] = raw.label.apply(lambda x: len(x.split(" "))//6-1)
  raw = raw[raw['n_boxes']>0].reset_index()

  gkf  = GroupKFold(n_splits = 5)
  raw['fold'] = -1
  for fold, (train_idx, val_idx) in enumerate(gkf.split(raw, groups = raw.id_image.tolist())):
      raw.loc[val_idx, 'fold'] = fold

  marking=pd.DataFrame(columns=['image_id', 'width', 'height', 'x', 'y', 'w', 'h','fold'])
  for _,row in tqdm(raw.iterrows(), total=len(raw)):
      bxs=np.array(row.label.split(" ")).reshape(-1,6)
      for bb in bxs:
          w_r=CFG.dim/row.width
          h_r=CFG.dim/row.height
          marking=marking.append({'image_id': row.id_image,
                                  'width':row.width,
                                  'height':row.height,
                                  'x':np.clip(float(bb[2]) * w_r,0,CFG.dim-1),
                                  'y':np.clip(float(bb[3]) * h_r,0,CFG.dim-1),
                                  'w':np.clip((float(bb[4])-float(bb[2])) * w_r,0,CFG.dim-1),
                                  'h':np.clip((float(bb[5])-float(bb[3])) * h_r,0,CFG.dim-1),
                                  'fold':row.fold}, ignore_index=True)
          
  train_df=marking[marking['fold']!=CFG.fold].reset_index(drop=True)
  valid_df=marking[marking['fold']==CFG.fold].reset_index(drop=True)

  print(train_df.head())

**Utility**

In [4]:
def gen_classes(CLASSES):
    classes = list()      
    
    for i, CLASS in enumerate(CLASSES):
        single_class = {} 
        single_class['id'] = i
        single_class['name'] = CLASS
        classes.append(single_class)
    return classes

def gen_objs(df, debug = False):
    
    if debug:
        
        df = df[:int(len(df)*0.05)]
        
    
    img_lists = list(df['image_id'].unique())
    imgs = list()
    objs = list()   
    
    ## gen images information
    for i in tqdm(range(len(img_lists))):
        '''
        
        I just notice that all images were preprocessed image size 720x1280
        If you want to check real image size, use this code below
        
        img = cv2.imread(os.path.join(data_path, img_lists[i]))
        
        single_img_obj = {}
        single_img_obj['file_name'] = img_lists[i]
        single_img_obj['height'] = img.shape[0]
        single_img_obj['width'] = img.shape[1]
        single_img_obj['id'] = i + 1
        '''
        
        single_img_obj = {}
        single_img_obj['file_name'] = img_lists[i] + '.png'
        single_img_obj['height'] = CFG.dim 
        single_img_obj['width'] = CFG.dim
        single_img_obj['id'] = i + 1        
        
        imgs.append(single_img_obj)
        
  
    ## gen objs information    
    for j in tqdm(range(len(df))):
        single_obj = {}
        single_obj['id'] = j + 1
        single_obj['image_id'] = img_lists.index(df['image_id'][j]) + 1
        single_obj['category_id'] = len(CFG.cls)-1
        single_obj['area'] = float(df['w'][j]*df['h'][j])
        single_obj['bbox'] = [int(df['x'][j]), int(df['y'][j]), int(df['w'][j]), int(df['h'][j])]
        single_obj['iscrowd'] = 0        
        
        objs.append(single_obj)

    print(f'images: {len(imgs)}, objs: {len(objs)}')
    
    return imgs, objs

def gen_coco(outpath, classes, objs, imgs, train=True):
    if train:
        data_dict = {}
        data_dict['images'] = []
        data_dict['annotations'] = []
        data_dict['categories'] = []
        data_dict['images'].extend(imgs)
        data_dict['annotations'].extend(objs)
        data_dict['categories'].extend(classes)
        
    else:
        data_dict = {}
        data_dict['images'] = []
        data_dict['categories'] = []
        
        data_dict['images'].extend(imgs)
        data_dict['categories'].extend(classes)   

    with open(outpath, 'w') as f_out:
        json.dump(data_dict, f_out)

In [5]:
import json
import copy

cls=gen_classes(CFG.cls)
print(cls)
if CFG.generate_ann:
  train_imgs, train_objs = gen_objs(train_df, CFG.debug)
  valid_imgs, valid_objs = gen_objs(valid_df, CFG.debug)
  gen_coco(CFG.prefix_ann+'train.json', cls, train_objs, train_imgs, train=True)
  gen_coco(CFG.prefix_ann+'valid.json', cls, valid_objs, valid_imgs, train=True)

[{'id': 0, 'name': 'opacity'}]


In [6]:
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector
import torch 

cfg = Config.fromfile(CFG._base_)
cfg.classes = CFG.cls

**SET CONFIG**

In [7]:
cfg.dataset_type = 'CocoDataset'

#TRAIN===============
cfg.dataset_type = 'CocoDataset'
cfg.data.train.ann_file = CFG.prefix_ann + 'train.json'
cfg.data.train.img_prefix = CFG.prefix_img
cfg.data.train.classes = cfg.classes

#VAL===============
cfg.dataset_type = 'CocoDataset'
cfg.data.val.ann_file = CFG.prefix_ann + 'valid.json'
cfg.data.val.img_prefix = CFG.prefix_img
cfg.data.val.classes = cfg.classes

#TEST===============
cfg.dataset_type = 'CocoDataset'
cfg.data.test.ann_file = CFG.prefix_ann + 'valid.json'
cfg.data.test.img_prefix = CFG.prefix_img
cfg.data.test.classes = cfg.classes


# modify num classes of the model in box head===============

#VfNet
cfg.model.bbox_head.num_classes=CFG.n_classes

#Cascade_Rcnn
#cfg.model.roi_head.bbox_head[0].num_classes=CFG.n_classes
#cfg.model.roi_head.bbox_head[1].num_classes=CFG.n_classes
#cfg.model.roi_head.bbox_head[2].num_classes=CFG.n_classes


# CheckPoint===============
cfg.load_from = CFG.checkpoint


#SEED===============
cfg.seed = CFG.seed
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


#EPOCHS===============
cfg.runner.max_epochs = CFG.epochs+1
cfg.total_epochs = CFG.epochs+1


# Set up working dir to save files and logs===============
path = CFG.outdir + CFG._base_.split("/")[-1] + f'-fold{CFG.fold}/'
os.makedirs(path, exist_ok = True)
cfg.work_dir = path


# The original learning rate (LR) is set for 8-GPU training===============
# We divide it by 8 since we only use one GPU===============
cfg.optimizer.lr = CFG.lr
cfg.lr_config = dict(
    policy='CosineAnnealing',
    warmup='linear',
    warmup_iters=1000,
    warmup_ratio=1.0 / 10,
    min_lr_ratio=CFG.min_lr)


# Change the evaluation metric since we use customized dataset===============
cfg.evaluation.metric = 'bbox'


# We can set the evaluation interval to reduce the evaluation times===============
cfg.evaluation.interval = 2
if CFG.debug:
  cfg.evaluation.interval = 1


# We can set the checkpoint saving interval to reduce the storage cost===============
cfg.checkpoint_config.interval = 2
if CFG.debug:
  cfg.checkpoint_config.interval= 1


#logging interval===============
cfg.log_config.interval = 100
if CFG.debug:
  cfg.log_config.interval = 10


# Set seed thus the results are more reproducible===============
cfg.seed = CFG.seed
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


**AUGMENTATION**

In [8]:
img_norm_cfg = dict(
    mean=[0, 0, 0], std=[255.0, 255.0, 255.0], to_rgb=True)


albu_train_transforms = [
                         
    dict(
        type='RandomResizedCrop',
        height=CFG.size,
        width=CFG.size,
        p=1),

    dict(
        type='RandomBrightnessContrast',
        brightness_limit=0.2,
        contrast_limit=0.2,
      p=0.75),

]


train_pipeline_a = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    #dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5, direction='horizontal'),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='coco',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='ImageToTensor'),
    dict(
        type='Collect',
        keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'],
        meta_keys=('filename', 'ori_shape', 'img_shape', 'img_norm_cfg',
                   'pad_shape', 'scale_factor')),
]


train_pipeline_n = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(CFG.size, CFG.size), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5, direction='horizontal'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='ImageToTensor'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]

In [9]:
cfg.train_pipeline=train_pipeline_a

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(CFG.size, CFG.size),
        flip=True,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip', direction='horizontal'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

cfg.data.samples_per_gpu=CFG.batch_size
cfg.data.workers_per_gpu=CFG.num_workers

In [10]:
print(f'Config: {cfg.pretty_text}')

Config: dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', flip_ratio=0.5, direction='horizontal'),
    dict(
        type='Albu',
        transforms=[
            dict(type='RandomResizedCrop', height=640, width=640, p=1),
            dict(
                type='RandomBrightnessContrast',
                brightness_limit=0.2,
                contrast_limit=0.2,
                p=0.75)
        ],
        bbox_params=dict(
            type='BboxParams',
            format='coco',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap=dict(img='image', gt_masks='masks', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Norm

In [11]:
model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/content/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-07-15 14:03:48,897 - mmdet - INFO - load checkpoint from https://openmmlab.oss-cn-hangzhou.aliyuncs.com/mmdetection/v2.0/vfnet/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth
2021-07-15 14:03:48,899 - mmdet - INFO - Use load_from_http loader


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


Downloading: "https://openmmlab.oss-cn-hangzhou.aliyuncs.com/mmdetection/v2.0/vfnet/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth" to /root/.cache/torch/hub/checkpoints/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth


2021-07-15 14:06:39,579 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.vfnet_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([1, 256, 3, 3]).
size mismatch for bbox_head.vfnet_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([1]).
unexpected key in source state_dict: backbone.layer3.6.conv1.weight, backbone.layer3.6.bn1.weight, backbone.layer3.6.bn1.bias, backbone.layer3.6.bn1.running_mean, backbone.layer3.6.bn1.running_var, backbone.layer3.6.bn1.num_batches_tracked, backbone.layer3.6.conv2.weight, backbone.layer3.6.conv2.conv_offset.weight, backbone.layer3.6.conv2.conv_offset.bias, backbone.layer3.6.bn2.weight, backbone.layer3.6.bn2.bias, backbone.layer3.6.bn2.running_mean, backbone.layer3.6.bn2.running_var, backbone.layer3.6.bn2.num_batches_tracked, backbone.layer3.6.conv3.weight, backbone.la

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 665/665, 11.8 task/s, elapsed: 57s, ETA:     0s

...

2021-07-15 17:08:05,150 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.24s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.36s).
Accumulating evaluation results...


2021-07-15 17:08:12,818 - mmdet - INFO - Epoch(val) [12][665]	bbox_mAP: 0.1970, bbox_mAP_50: 0.5510, bbox_mAP_75: 0.0890, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.0420, bbox_mAP_l: 0.2260, bbox_mAP_copypaste: 0.197 0.551 0.089 0.000 0.042 0.226


DONE (t=1.67s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.197
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.551
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.042
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.226
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.502
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.502
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.502
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.352
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.534


2021-07-15 17:10:26,117 - mmdet - INFO - Epoch [13][100/664]	lr: 4.394e-05, eta: 0:50:51, time: 1.331, data_time: 0.035, memory: 5552, loss_cls: 0.8311, loss_bbox: 0.4297, loss_bbox_rf: 0.5721, loss: 1.8329
2021-07-15 17:12:40,399 - mmdet - INFO - Epoch [13][200/664]	lr: 4.394e-05, eta: 0:48:56, time: 1.343, data_time: 0.013, memory: 5552, loss_cls: 0.8277, loss_bbox: 0.4309, loss_bbox_rf: 0.5742, loss: 1.8328
2021-07-15 17:14:52,293 - mmdet - INFO - Epoch [13][300/664]	lr: 4.394e-05, eta: 0:47:00, time: 1.319, data_time: 0.013, memory: 5552, loss_cls: 0.8261, loss_bbox: 0.4370, loss_bbox_rf: 0.5817, loss: 1.8449
2021-07-15 17:17:03,824 - mmdet - INFO - Epoch [13][400/664]	lr: 4.394e-05, eta: 0:45:03, time: 1.315, data_time: 0.013, memory: 5552, loss_cls: 0.8230, loss_bbox: 0.4336, loss_bbox_rf: 0.5774, loss: 1.8339
2021-07-15 17:19:13,407 - mmdet - INFO - Epoch [13][500/664]	lr: 4.394e-05, eta: 0:43:06, time: 1.296, data_time: 0.013, memory: 5552, loss_cls: 0.8232, loss_bbox: 0.4270, 